### Import Dependencies

In [8]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import json
import cv2
import numpy as np

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Create Torch Dataset

In [11]:
class KeypointsDataset(Dataset):
    def __init__(self, img_dir, json_file):
        self.img_dir = img_dir
        with open(json_file, "r") as f:
            self.data = json.load(f)
        
        self.transforms = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])            
        ])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.img_dir}/{item['id']}.png")
        h, w = img.shape[:2]
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(img)
        
        kps = np.array(item["kps"]).flatten()
        kps = kps.astype(np.float32)
        
        kps[::2] *= 224.0 / w
        kps[1::2] *= 224.0 / h
        
        return img, kps

In [12]:
train_dataset = KeypointsDataset("tennis_court_det_dataset/data/images", "tennis_court_det_dataset/data/data_train.json")
train_loader = DataLoader(train_dataset, batch_size = 8, shuffle = True)

val_dataset = KeypointsDataset("tennis_court_det_dataset/data/images", "tennis_court_det_dataset/data/data_val.json")
val_loader = DataLoader(val_dataset, batch_size = 8, shuffle = False)

### Create Model

In [13]:
model = models.resnet50(pretrained = True)
model.fc = torch.nn.Linear(model.fc.in_features, 14 * 2)
# model.load_state_dict(torch.load("models/tennis_court_keypoints_resnet50_30_epochs.pth", map_location = 'cuda'))
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\xiani/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

### Train Model

In [14]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

epochs = 20
for epoch in range(epochs):
    for i, (img, kps) in enumerate(train_loader):
        img = img.to(device)
        kps = kps.to(device)
        
        optimizer.zero_grad()
        pred = model(img)
        loss = criterion(pred, kps)
        loss.backward()
        optimizer.step()
        
    print(f"Epoch: {epoch}, Loss: {loss.item()}")

c:\Users\xiani\anaconda3\envs\tf_gpu\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0, Loss: 352.0243225097656
Epoch: 1, Loss: 26.88661766052246
Epoch: 2, Loss: 18.098644256591797
Epoch: 3, Loss: 11.040696144104004
Epoch: 4, Loss: 9.440413475036621
Epoch: 5, Loss: 10.949520111083984
Epoch: 6, Loss: 16.253503799438477
Epoch: 7, Loss: 1.7197483777999878
Epoch: 8, Loss: 2.3674914836883545
Epoch: 9, Loss: 4.402019023895264
Epoch: 10, Loss: 10.648868560791016
Epoch: 11, Loss: 0.7475415468215942
Epoch: 12, Loss: 3.734015464782715
Epoch: 13, Loss: 100.0947265625
Epoch: 14, Loss: 1.6537269353866577
Epoch: 15, Loss: 7.490461826324463
Epoch: 16, Loss: 5.543719291687012
Epoch: 17, Loss: 38.37974548339844
Epoch: 18, Loss: 0.7960019707679749
Epoch: 19, Loss: 1.3446208238601685


### Save Model

In [15]:
# torch.save(model.state_dict(), "models/tennis_court_keypoints_resnet50_40_epochs.pth")